# TUTORIAL
#### Hi! First time here? You are in the right place to learn how to use the code 

In [1]:
using GW

In [2]:
# these three packages are needed only for the tutorial
using Test
using BenchmarkTools
using Base.Threads
nthreads()

8

In [3]:
# if the previous cell returns 1, you need to change your .bashrc file to include the following line:
# export JULIA_NUM_THREADS = the_number_of_threads_of_your_cpu (e.g.  JULIA_NUM_THREADS = 8) then restart the terminal (or use the command source ~/.bashrc)

## FIRST PART: the basics

#### In this first part we will calculate the SNR, the Fisher Matrix and invert it, finally we will obtain the errors estimates.
#### But first we need to decide a waveform model and a detector configuration

In [4]:
# check the waveforms available with this function.
# In the code the functions are indicated with the name of the module in which they are contained, a dot and the name of the function
# There are 4 modules: waveform, detector, catalog and UtilsAndConstants (called with uc)
_available_waveforms()
# TaylorF2 is a very basic waveform, not useful
# PhenomD is a good compromise between accuracy and speed for this tutorial. Valid only for BBH
# PhenomHM is the most accurate waveform, but it is also the slowest. The recommended one for the final results, valid only for BBH
# PhenomD_NRTidal is the PhenomD waveform with the addition of the tidal effects. Valid only for BNS
# PhenomNSBH is the waveform valid for NSBH

5-element Vector{String}:
 "TaylorF2"
 "PhenomD"
 "PhenomHM"
 "PhenomD_NRTidal"
 "PhenomNSBH"

##### The main functions are indicated with the capital letter.
##### The other functions are auxiliary functions, indicated with the underscore.

In [5]:
# check the detectors available with this function
_available_detectors()
# the length in km is the arm length of the detector and it is not needed to call the detector
# e.g. _available_detectors("CE1Id")

11-element Vector{String}:
 "CE1Id, 40km"
 "CE2NM, 20km"
 "CE2NSW, 20km"
 "ETS, 10km"
 "ETLS, 10km"
 "ETMR, 10km"
 "ETLMR, 10km"
 "LIGO_L"
 "LIGO_H"
 "VIRGO"
 "KAGRA"

In [6]:
# now we can decide the waveform, remember the parenthesis (they are needed to define the model structure)

wfPhenomD = PhenomD()
# or use the function _available_waveforms()
wfPhenomD = _available_waveforms("PhenomD")

# then decide the detector
CE_1= CE1Id
# or use the function _available_detectors()
CE_1 = _available_detectors("CE1Id");

# to create a network of detectors put them in an array

CE_2 = CE2NM
ET = ETS

network = [CE_1, CE_2, ET]

3-element Vector{Detector}:
 Detector(0.7649254512715546, -1.9691677285626024, -0.7853981633974483, 1.5707963267948966, 'L', [5.000000000000001, 5.034693157380139, 5.069627037794076, 5.104803311530235, 5.140223660466547, 5.175889778150882, 5.211803369882009, 5.247966152791138, 5.284379855924022, 5.321046220323622  …  4698.324157477345, 4730.924097361, 4763.750236213645, 4796.804143546573, 4830.087399761323, 4863.601596225268, 4897.348335347697, 4931.32923065641, 4965.5459068749, 4999.999999999999], [5.320053993725279e-47, 4.977802675091629e-47, 4.663048207778976e-47, 4.373069475460877e-47, 4.1054566210234874e-47, 3.8580723586682724e-47, 3.6290183152679144e-47, 3.416605735864509e-47, 3.219329997795618e-47, 3.035848361932253e-47  …  1.1542927889652714e-48, 1.1798913424045747e-48, 1.206559811213688e-48, 1.2342718680182324e-48, 1.2630057938779226e-48, 1.2927441289587634e-48, 1.3234733704060701e-48, 1.3551836909058734e-48, 1.387868689337051e-48, 1.421525176464475e-48], "CE1Id")
 Detector(0.

In [7]:
# Let's calculate the SNR of an event with a given waveform and detector

# first we need to define the parameters of the event
mc = 30.0 # chirp mass at the detector frame in solar masses
η = 0.2 # symmetric mass ratio, dimensionless 
χ_1 = 0.1 # dimensionless spin of the most massive BH
χ_2 = 0.2 # dimensionless spin of the least massive BH
dL = 1.0 # luminosity distance in Gpc
θ = 45. * pi/180 # inclination angle in radians
ϕ = 45. * pi/180 # polarization angle in radians
ι = 45. * pi/180 # inclination angle in radians
ψ = 45. * pi/180 # polarisation angle in radians
tcoal = 0.0 # time of coalescence in fraction of a day, [0,1]
Φ_coal = 0.0; # phase of coalescence in radians

parameters = [mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal];

In [8]:
snr = SNR(wfPhenomD, mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, network)
# the function returns the SNR of the event, the order of parameters is very important in the code
# first there is the waveform, then there are the intrinsic parameters of the event, then the extrinsic parameters and finally the detector
# in particular, after the waveform, start with the chirp mass, then the symmetric mass ratio, then the spins, then the luminosity distance,
# then the angles, then the time of coalescence and finally the phase of coalescence

# the code requires the least amount of information, so in the SNR there is no need for the phase to coalescence,
# since the SNR depends only on the amplitude, which is independent of the phase.
# Thus the function SNR gives an error if you put also the phase of coalescence

777.6094582520611

In [9]:
# check that you obtained the correct result
snr == 777.6094582520611    # true


true

In [10]:
# or to be fancier, use the Test package
@test snr == 777.6094582520611

Test Passed

In [11]:
#check with the snr calculated by another code (GWFAST)
snrGWFAST = 777.60945825
isapprox(snr,snrGWFAST, rtol = 1e-10)    # true with relative tolerance 1e-10

true

In [12]:
# We can now move to the Fisher matrix of that event but choose a network of detectors
myfisher = FisherMatrix(wfPhenomD, mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal, network, coordinate_shift = false)
# It should take a minute or so because the code needs to be compiled first, each subsequent call will be blitzing fast
# to simplify the notation you can also write 
#myfisher = FisherMatrix(wfPhenomD, parameters..., network)

# the option coordinate_shift is related to T detectors (ET in this case) and represents the fact that the three detectors inside ET are not in the exact same position.
# This produces a difference in phase that improves the estimation of the parameters, but this option is not implemented in GWFAST, so we put it false to do a comparison.
# The default is true, so if you want to use it, you can just remove the option.

# the fisher is a 11x11 matrix (in the BBH case), the order is the same as the inputs.

# the units of the fisher matrix are the inverse of the square of the units of the parameters
# in particular the units of the chirp mass are solar masses, the units of the symmetric mass ratio are dimensionless, the units of the spins are dimensionless,
# the units of the luminosity distance are Gpc, the units of the angles are radians, the units of the time of coalescence are fraction of a day and the units of the phase of coalescence are radians.
# The errors keep the same units except for the time of coalescence, which is converted to seconds.

11×11 Matrix{Float64}:
      1.25029e10  -4.95201e10      -1.29424e10  …  -2.00353e10      8.67874e7
     -4.95201e10   2.02027e11       5.24711e10      9.19228e10     -3.40806e8
     -1.29424e10   5.24711e10       1.36461e10      2.32925e10     -8.92274e7
     -3.61225e9    1.44675e10       3.77269e9       5.85595e9      -2.4962e7
 -15025.3         -6.28205e5   -70159.6            -0.163545        0.0
     -1.44548e8    6.57912e8        1.67127e8   …   4.32171e8      -9.64147e5
     -2.43649e8    1.03555e9        2.66514e8       6.58424e8      -1.66962e6
      4.2846e6    -1.82023e7       -4.64556e6      -6.23763e6   29225.0
      1.74589e8   -6.85794e8       -1.79541e8      -2.71056e8       1.21621e6
     -2.00353e10   9.19228e10       2.32925e10      7.69501e10     -1.34815e8
      8.67874e7   -3.40806e8       -8.92274e7   …  -1.34815e8       6.04676e5

In [13]:
# this is a comparison of the Fisher matrix calculated by the code and the Fisher matrix calculated by GWFAST, we only compare one row but the result is valid for the whole matrix (in the limit of machine precision)
fisher_first_row_GWFAST = [1.25028777e+10, -4.95201244e+10, -1.29423577e+10, -3.61224785e+09, -1.50253200e+04, -1.44547812e+08, -2.43648573e+08, 4.28460151e+06, 1.74588867e+08, -2.00352948e+10,
                        8.67874125e+07]
isapprox(myfisher[1,:], fisher_first_row_GWFAST, rtol = 1e-8)    # true with relative tolerance 1e-8


true

In [14]:
# we can then calculate the covariance matrix by inverting the Fisher matrix (uc refers to the utils and constants module)
mycovariance = CovMatrix(myfisher)

11×11 Matrix{Float64}:
  7.23242e-7  -3.2975e-7    1.08466e-6   …   3.0717e-8   -3.32002e-5
 -3.2975e-7    6.54296e-7  -1.1187e-5        4.10002e-7   1.26942e-5
  1.08466e-6  -1.1187e-5    0.000253739     -1.02455e-5  -0.00012387
  2.16537e-6   2.7996e-5   -0.000698006      2.89052e-5   0.000192447
 -1.01296e-7   5.55902e-7  -1.14271e-5       4.84782e-7  -9.02915e-6
 -7.56714e-9   6.4974e-9   -1.96666e-7   …   5.14175e-9  -3.3578e-7
 -1.01271e-8   2.18286e-8  -2.66614e-7      -1.61353e-8   8.01447e-6
 -2.95392e-8  -2.48429e-7   5.97057e-6      -2.87899e-7   5.39887e-6
 -1.00872e-7  -7.77319e-8   2.4162e-6       -9.98621e-8  -0.000267982
  3.0717e-8    4.10002e-7  -1.02455e-5       4.24692e-7   2.94364e-6
 -3.32002e-5   1.26942e-5  -0.00012387   …   2.94364e-6   0.00280405

In [15]:
# we can now calculate the errors on the parameters
myerrors = Errors(mycovariance)
parameters_string = ["mc", "η", "χ_1", "χ_2", "dL", "θ", "ϕ", "ι", "ψ", "tcoal", "Φ_coal"]

for i in eachindex(myerrors)
    println("The error on $(parameters_string[i]) is $(myerrors[i])")
end

The error on mc is 0.0008504360749730822
The error on η is 0.0008088856061285265
The error on χ_1 is 0.01592917387229465
The error on χ_2 is 0.044367410029956963
The error on dL is 0.007751205023032693
The error on θ is 0.0004336067649812443
The error on ϕ is 0.0019172151420981257
The error on ι is 0.009008641014767917
The error on ψ is 0.011691407428797717
The error on tcoal is 0.0006516841768254732
The error on Φ_coal is 0.05295327234454468


In [16]:
# check the values of the errors with the values calculated by GWFAST
errors_GWFAST = [0.00085044, 0.00080889 , 0.01592917, 0.04436741, 0.00775121, 0.00043361, 0.00191722, 0.00900864, 0.01169141, 0.00065168, 0.05295327]
isapprox(myerrors, errors_GWFAST, rtol = 1e-6)    # true with relative tolerance 1e-6


true

In [17]:
# obtaint the sky area of the event
sky_area = SkyArea(mycovariance, θ) # the angle is in grad and represents the 90% credible interval
# for different credible intervals you can use the function uc.SkyArea(mycovariance, θ, credible_interval = 0.68) or 0.95

0.024299733801812353

### This is the end of the first part of the tutorial
#### In the second part we will dig a bit deeper into the code and see how to use the catalog module to generate a catalog of events

In [18]:
#################################################################################################################################################################################

# Second part: generate a catalog, the detector structure and loops
### The detector structure

In [19]:
# In the next cells we will go through the detector structure in the code,
# if you will use the detectors already present in the code you can skip this part.
# A detector in the code is a structure with the following fields

fieldnames(typeof(CE_1))


(:latitude_rad, :longitude_rad, :orientation_rad, :arm_aperture_rad, :shape, :fNoise, :psd, :label)

In [20]:
# to see what each field contains, use the following loop
for field in fieldnames(typeof(CE_1))
    println("$(field): $(getfield(CE_1, field))")
end

latitude_rad: 0.7649254512715546
longitude_rad: -1.9691677285626024
orientation_rad: -0.7853981633974483
arm_aperture_rad: 1.5707963267948966
shape: L
fNoise: [5.000000000000001, 5.034693157380139, 5.069627037794076, 5.104803311530235, 5.140223660466547, 5.175889778150882, 5.211803369882009, 5.247966152791138, 5.284379855924022, 5.321046220323622, 5.357966999113357, 5.395143957580921, 5.432578873262691, 5.4702735360287145, 5.508229748168284, 5.546449324476114, 5.584934092339116, 5.623685891823759, 5.66270657576406, 5.701998009850165, 5.7415620727175565, 5.7814006560368805, 5.821515664604383, 5.861909016432995, 5.902582642844027, 5.9435384885595175, 5.984778511795218, 6.026304684354212, 6.068118991721205, 6.110223433157443, 6.1526200217963085, 6.195310784739582, 6.238297763154348, 6.281583012370604, 6.325168601979518, 6.369056615932394, 6.4132491526403035, 6.457748325074419, 6.5025562608670455, 6.547675102413337, 6.593107006973743, 6.63885414677715, 6.684918709124732, 6.731302896494551,

In [21]:
# If you want to define your detector from scratch you can do it like

my_detector = Detector(43.827 * pi/180,
                                 -112.825 * pi/180,
                                  -45 * pi/180,
                                   90. * pi/180,
                                    'L',
                                 _readASD("useful_files/psds/ce_strain/cosmic_explorer.txt")...,
                                  "mydetector"
                                ) # in julia "" and '' are NOT equivalent, "s" is a string, 's' is a character

# the function _readASD reads the asd (amplitude spectral density) and outputs the psd (power spectral density) 
# It reads from a file which must have at least two columns, the first is the frequency and the second the asd
# if the file has more columns, to read other columns you can add them as arguments of the function _readASD("path", col=[1,3]) for example to read the first and third column
# the function returns the frequency and the psd, so you can use the splat operator (the dots) to pass the arguments to the Detector structure

# the detector structure is mutable, so you can change the values of the fields as you like, but remember that the fields are in the order of the constructor and you need to initialize all of them


Detector(0.7649254512715546, -1.9691677285626024, -0.7853981633974483, 1.5707963267948966, 'L', [5.000000000000001, 5.034693157380139, 5.069627037794076, 5.104803311530235, 5.140223660466547, 5.175889778150882, 5.211803369882009, 5.247966152791138, 5.284379855924022, 5.321046220323622  …  4698.324157477345, 4730.924097361, 4763.750236213645, 4796.804143546573, 4830.087399761323, 4863.601596225268, 4897.348335347697, 4931.32923065641, 4965.5459068749, 4999.999999999999], [5.320053993725279e-47, 4.977802675091629e-47, 4.663048207778976e-47, 4.373069475460877e-47, 4.1054566210234874e-47, 3.8580723586682724e-47, 3.6290183152679144e-47, 3.416605735864509e-47, 3.219329997795618e-47, 3.035848361932253e-47  …  1.1542927889652714e-48, 1.1798913424045747e-48, 1.206559811213688e-48, 1.2342718680182324e-48, 1.2630057938779226e-48, 1.2927441289587634e-48, 1.3234733704060701e-48, 1.3551836909058734e-48, 1.387868689337051e-48, 1.421525176464475e-48], "mydetector")

### Catalogs

In [22]:
# To create a catalog you need to decide the  number of events and the population (BBH, BNS or NSBH)
# this function writes the catalog as a .h5 file in the folder catalogs/
# the function returns the parameters of the events in the catalog
nEvents = 1_000
name = "my_first_catalog.h5"
typeof_events = "BBH"
mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal = GenerateCatalog(nEvents, typeof_events, name_catalog = name)


([21.751979078202886, 12.069837020267995, 29.723691336020863, 41.4746296374311, 21.946745346558043, 8.367026920159512, 20.865822409058094, 54.88058443177169, 17.20806653088661, 32.625537967260996  …  20.56248369004362, 59.22513387981316, 27.103046516516212, 36.175479789368616, 18.25923297681047, 28.26937776963734, 36.20946443432673, 17.7886099594234, 15.020231626123199, 27.708058709325215], [0.24957863143743464, 0.2345206878941768, 0.24990984247656356, 0.24982962853949064, 0.2499822492709117, 0.2499651363490702, 0.2484218243932461, 0.24569933298443736, 0.24989785642177847, 0.17392453374726824  …  0.2486257199403904, 0.24990603433291728, 0.24998478446696, 0.2494362314050022, 0.24817069225504526, 0.24999948985855994, 0.15342513324675616, 0.24996790814129985, 0.24999597656934275, 0.24860629549699842], [0.25324160625940223, 0.3731952773932649, 0.31617866678573214, 0.19781949634890109, 0.10619034140220865, 0.6125419669977715, 0.09228948845540375, 0.15942080155913904, 0.13390794751502783, 0.

In [23]:
### After you create the catalog you can read it with the following function
# note that is also prints some useful information about the catalog contained in the .h5 file
mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal = ReadCatalog(name)

Attributes: ["SFR", "format", "number_events", "population", "seed", "time_delay_in_Myrs", "total_number_sources_yr"]
SFR: Madau&Dickinson
format: GWJulia
number_events: 1000
population: BBH
seed: 8270
time_delay_in_Myrs: 10.0
total_number_sources_yr: 35787.84461570478
Parameters: ["Lambda1", "Lambda2", "chi1", "chi2", "dL", "eta", "iota", "mc", "phi", "phiCoal", "psi", "tcoal", "theta", "z"]


([21.751979078202886, 12.069837020267995, 29.723691336020863, 41.4746296374311, 21.946745346558043, 8.367026920159512, 20.865822409058094, 54.88058443177169, 17.20806653088661, 32.625537967260996  …  20.56248369004362, 59.22513387981316, 27.103046516516212, 36.175479789368616, 18.25923297681047, 28.26937776963734, 36.20946443432673, 17.7886099594234, 15.020231626123199, 27.708058709325215], [0.24957863143743464, 0.2345206878941768, 0.24990984247656356, 0.24982962853949064, 0.2499822492709117, 0.2499651363490702, 0.2484218243932461, 0.24569933298443736, 0.24989785642177847, 0.17392453374726824  …  0.2486257199403904, 0.24990603433291728, 0.24998478446696, 0.2494362314050022, 0.24817069225504526, 0.24999948985855994, 0.15342513324675616, 0.24996790814129985, 0.24999597656934275, 0.24860629549699842], [0.25324160625940223, 0.3731952773932649, 0.31617866678573214, 0.19781949634890109, 0.10619034140220865, 0.6125419669977715, 0.09228948845540375, 0.15942080155913904, 0.13390794751502783, 0.

### SNRs with for loops and multi-threading

In [24]:
# calculate the SNR for 100 events

snrs = Vector{Float64}(undef, nEvents) # create a vector to store the snrs

@btime for ii in 1:nEvents  # SNR of the first 100 events, the @btime macro is used to measure the time of the operation
    snrs[ii]=SNR(wfPhenomD, mc[ii], η[ii], χ_1[ii], χ_2[ii], dL[ii], θ[ii], ϕ[ii], ι[ii], ψ[ii], tcoal[ii], network)
end


  2.948 s (9079979 allocations: 1.20 GiB)


In [25]:
# if the number of events is larger than 100 we suggest to go multi-thread (up to now everything was done with a single thread)
# check the number of threads available with
nthreads()


8

In [26]:
# calculate the SNR, this time with multi-threading 

snrs = Vector{Float64}(undef, nEvents)
@btime @threads for ii in 1:nEvents # the @threads macro is used to parallelize the for loop
    snrs[ii]=SNR(wfPhenomD, mc[ii], η[ii], χ_1[ii], χ_2[ii], dL[ii], θ[ii], ϕ[ii], ι[ii], ψ[ii], tcoal[ii], network)
end

# the time should be smaller than the previous one, if not, try to increase the number of sources

  673.218 ms (9083425 allocations: 1.20 GiB)


In [27]:
# All that we did so far can be automatized with the SNR function
# we just need to pass all the parameter to the SNR function, it will already parallelize the calculation
# it will return the SNRs and if you use the flag "auto_save=true" it will save them in a file called SNRs.h5 in the folder output/BBH
# to change the destination folder you can use the flag "name_folder = "path/to/folder""

snrs = SNR(wfPhenomD, mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, network, auto_save=true);
# the output is a vector with the SNRs of the events

Computing SNRs... 100%|██████████████████████████████████| Time: 0:00:01


SNRs computed!
The evaluation took: 1.460685791 seconds.


1000-element Vector{Float64}:
  21.843287256466215
 129.26950846769807
  24.099974416048425
  30.841904423278727
  25.132585349239154
 278.33082570792806
  23.410609400550594
  13.929177234133075
  41.531000301724646
  31.508787389866217
   ⋮
  42.871264485407565
  60.04397052138793
  14.383301046929159
  41.091006205842476
  13.85651105104916
  40.74080444619697
  28.14679611029475
  28.22538859563064
  20.817121479557233

### Fisher matrix with for loops and multi-threading

In [28]:
# We can reproduce the steps we did before for the SNRs

nPar = 11 # number of parameters of the binary system for BBH
# to speed things up we do not calculate the Fisher matrix for all the events, but only for the first 100
Fisher = Array{Float64}(undef, 100, nPar, nPar)   # create a 3D array to store the Fisher matrices

@time for ii in 1:100
    Fisher[ii,:,:] = FisherMatrix(wfPhenomD, mc[ii], η[ii], χ_1[ii], χ_2[ii], dL[ii], θ[ii], ϕ[ii], ι[ii], ψ[ii], tcoal[ii], Φ_coal[ii], network)
end

# here we use the macro @time to measure the time of the operation
# the time takes also into account the compilation time, so the first time you run the code it will be longer than the subsequent times
# instread @btime does more runs of the code to give a more accurate estimate of the time

  8.266108 seconds (22.02 M allocations: 5.976 GiB, 4.48% gc time, 12.28% compilation time)


In [29]:
# As before we can go multi-threading with the macro @threads

Fisher = Array{Float64}(undef, 100, nPar, nPar)
@time @threads for i in 1:100
    Fisher[i,:,:] = FisherMatrix(wfPhenomD, mc[i], η[i], χ_1[i], χ_2[i], dL[i], θ[i], ϕ[i], ι[i], ψ[i], tcoal[i], Φ_coal[i], network)
end


 10.033945 seconds (35.91 M allocations: 6.821 GiB, 7.80% gc time, 591.74% compilation time)


In [30]:
# To do everything together we can use the function FisherMatrix passing directly the parameters of the events
# the function will calculate the Fisher matrix for all the events and save them in a file called fisher.h5 in the folder output/BBH
# to change the destination folder you can use the flag "folder = "path/to/folder""
# the function will also return the Fisher matrices
# the function calculates also the SNRs, so you do not need to calculate them before
# if you want to return the SNRs use the flag "return_SNR = true"
# if you save the Fisher matrices you also save the SNRs if the return_SNR flag is true 

Fisher, SNRs = FisherMatrix(wfPhenomD, mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal, network, return_SNR = true, auto_save=true);

# the output of the FisherMatrix function is a 3D array with the Fisher matrices of the events and has the same structure as the Fisher array created
# before. Thus Fisher[1,:,:] is the Fisher matrix of the first event, Fisher[:,1,1] is the first element of the Fisher matrix of all the events and so on.

Computing Fishers and SNRs... 100%|██████████████████████| Time: 0:00:18


Fisher matrices and SNRs computed!
The evaluation took: 18.420572644 seconds.


([5.76029105001422e7 -3.0753161232369345e7 … -4.25569917033818e7 165709.7445912737; 4.538236522353339e10 -9.54299932398243e9 … -8.167121441784571e9 2.7500785020331353e7; … ; 6.882142525090446e8 -1.4552850348783745e10 … -1.9699551590337247e8 740086.2134856242; 1.3913397140453495e7 -2.5727147386583127e7 … -2.2327914127940714e7 77535.57690551887;;; -3.0753161232369345e7 1.680191176618366e7 … 2.5548721396157913e7 -88134.07500479491; -9.54299932398243e9 2.1070612935127013e9 … 2.132810870348174e9 -5.734474228132469e6; … ; -1.4552850348783745e10 3.1233322264427484e11 … 4.723632236495142e9 -1.5604137243304787e7; -2.5727147386583127e7 4.9296435577841744e7 … 4.920361921924308e7 -142075.69505241964;;; -1.7526105361837056e7 9.493380509795485e6 … 1.4031979361831121e7 -50298.864595487285; -5.257270996422515e9 1.1366208227093344e9 … 1.0668689574255424e9 -3.170059237837784e6; … ; -9.321283228543223e7 1.9943502798429847e9 … 2.9058554030564863e7 -99985.45045590476; -7.08327650747853e6 1.343604408790972e

In [31]:
# to add the correction due to Earth motion just add the flag "useEarthMotion = true"

Fisher, SNRs = FisherMatrix(wfPhenomD, mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal, network, return_SNR = true, auto_save=true, useEarthMotion = true);


Computing Fishers and SNRs... 100%|██████████████████████| Time: 0:00:41


Fisher matrices and SNRs computed!
The evaluation took: 41.049985242 seconds.


##### Changing the SNR threshold

In [32]:
# the SNRs are calculated before the Fisher because under a threshold the Fisher matrix is not calculated, so the SNRs are needed to filter the events.
# The threshold is set to 12, but you can change it with the flag "rho_thres = value", e.g. rho_thres = 10.
# if you want to calculate the Fisher matrix for all the events, you can set the threshold to nothing, "rho_thres = nothing",
# this way it will not calculate the SNRs

Fisher_no_thres = FisherMatrix(wfPhenomD, mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal, network, rho_thres = nothing);


Computing Fishers... 100%|███████████████████████████████| Time: 0:00:16


Fisher matrices computed!
The evaluation took: 16.728181709 seconds.


In [33]:
# if an event is under the threshold, the Fisher matrix is not calculated and the code returns a matrix of zeros
# this choice was made to keep track of which event correspond to which Fisher matrix.
# To select only the events above the threshold just type

Fisher_above_thres = Fisher[Fisher[:,1,1].!=0,:,:];
println("The number of events above the threshold is $(size(Fisher_above_thres)[1])")


The number of events above the threshold is 975


In [34]:
# To read the Fisher matrices and the SNRs you can use the following function
Fishers, SNRs = _read_Fishers_SNRs("output/BBH/Fishers_SNRs.h5")

Attributes: ["Detectors", "What_this_file_contains", "number_events"]
Detectors: ["CE1Id", "CE2NM", "ETS"]
What_this_file_contains: This file contains the Fishers and the SNRs for 1000 events obtained with the PhenomD waveform model. The calculations are performed with the CE1Id,CE2NM,ETS detectors and the correction due to Earth Motion was true.
number_events: 1000
Keys: ["Fishers", "SNRs"]


([5.760808360950134e7 -3.0755285730977084e7 … -4.255813076541887e7 165725.61397053278; 4.5433225720178474e10 -9.550401776810207e9 … -8.169132427615334e9 2.7534652980938274e7; … ; 6.880285427239302e8 -1.4549782471797567e10 … -1.9698301762603456e8 739862.6364444281; 1.3915638021553405e7 -2.5730592585997738e7 … -2.2328838982118893e7 77548.59854365721;;; -3.0755285730977084e7 1.6802812469564125e7 … 2.5549221418015257e7 -88140.50861812204; -9.550401776810207e9 2.1081850953976326e9 … 2.133129347333427e9 -5.739306295327415e6; … ; -1.4549782471797567e10 3.1228179939218805e11 … 4.72341561463537e9 -1.5600488871837674e7; -2.5730592585997738e7 4.930182585309873e7 … 4.920510518277828e7 -142095.54234972273;;; -1.752744937772844e7 9.493942839760762e6 … 1.4032286962170612e7 -50302.95592952305; -5.262079769313509e9 1.1373374043488903e9 … 1.067068180222033e9 -3.1732252068118057e6; … ; -9.31926787575906e7 1.9940127336787858e9 … 2.9057134950015806e7 -99961.47051013532; -7.08427733594499e6 1.34376030496332

In [35]:
# To extract the covariance do as before but with for loops
nPar = 11
covs = Array{Float64}(undef, nEvents, nPar, nPar)
for ii in 1:nEvents
    covs[ii,:,:] = CovMatrix(Fisher[ii,:,:])
end
# if the covariance matrix is singular, the function will return a matrix with zeros, this will happen if the event is poorly 
# resolved (usually in the case of one or two detectors). In this case, the code will also return a warning message, one message for each event.

In [36]:
# Then to extract the errors
errors = Array{Float64}(undef, nEvents, nPar)
for ii in 1:nEvents
    errors[ii,:] = Errors(covs[ii,:,:])
end

In [37]:
# To extract the sky area then
sky_areas = Vector{Float64}(undef, nEvents)
for i in 1:nEvents
    sky_areas[i] = SkyArea(covs[i,:,:], θ[i])
end

In [38]:
# through out the tutorial we inizialed the empty arrays with undef, but you can also initialize them with zeros
# e.g. covs = zeros(nEvents, nPar, nPar)

In [39]:
#################################################################################################################################################################################

## Third Part: BNS and NSBH

In [40]:
# We do as before but this time we need to include the deformabilities of the neutron stars
# Λ_1 and Λ_2, they are dimensionless, in the case of BBH they are set to 0.
# The function GenerateCatalog will generate the catalog with the deformabilities

nEvents = 1_000
name = "my_first_catalog_of_BNS.h5"
typeof_events = "BNS"
mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal, Λ_1, Λ_2 = GenerateCatalog(nEvents, typeof_events, name_catalog = name)

([2.105434645156427, 5.302613180849204, 1.975763784836993, 3.640789469931674, 1.8937398688866058, 2.7924322976302305, 3.218326607498097, 4.110661429505073, 7.970601103084561, 2.2868356161141055  …  5.857123970977761, 4.566093870085379, 6.713753592580328, 2.425448481547478, 7.496930163930859, 3.5618402735192016, 1.7260519142961381, 3.081108565294565, 4.0261574781388285, 5.369587897896572], [0.24991243106443875, 0.237255662790869, 0.22909923173868516, 0.24999901346614606, 0.23442770542064947, 0.24847991442943457, 0.22875071752201975, 0.24198364218073104, 0.24198840989555204, 0.24789728940228822  …  0.2472654324777725, 0.24994849823027987, 0.24965556489506444, 0.24110548840046703, 0.2254150756298932, 0.24214012031787954, 0.24859331713001928, 0.24039556770542192, 0.24328840015335387, 0.24999021734633503], [-0.04226101813275727, 0.04274789309989058, 0.009490517261714827, 0.011269530514800695, -0.0034061250831855516, -0.029007677827829093, 0.04665914067479125, 0.022673213016811397, 0.0006165

In [41]:
# we can read the catalog as before
mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal, Λ_1, Λ_2 = ReadCatalog(name)

Attributes: ["SFR", "format", "number_events", "population", "seed", "time_delay_in_Myrs", "total_number_sources_yr"]
SFR: Madau&Dickinson
format: GWJulia
number_events: 1000
population: BNS
seed: 4998
time_delay_in_Myrs: 10.0
total_number_sources_yr: 222095.15335040318
Parameters: ["Lambda1", "Lambda2", "chi1", "chi2", "dL", "eta", "iota", "mc", "phi", "phiCoal", "psi", "tcoal", "theta", "z"]


([2.105434645156427, 5.302613180849204, 1.975763784836993, 3.640789469931674, 1.8937398688866058, 2.7924322976302305, 3.218326607498097, 4.110661429505073, 7.970601103084561, 2.2868356161141055  …  5.857123970977761, 4.566093870085379, 6.713753592580328, 2.425448481547478, 7.496930163930859, 3.5618402735192016, 1.7260519142961381, 3.081108565294565, 4.0261574781388285, 5.369587897896572], [0.24991243106443875, 0.237255662790869, 0.22909923173868516, 0.24999901346614606, 0.23442770542064947, 0.24847991442943457, 0.22875071752201975, 0.24198364218073104, 0.24198840989555204, 0.24789728940228822  …  0.2472654324777725, 0.24994849823027987, 0.24965556489506444, 0.24110548840046703, 0.2254150756298932, 0.24214012031787954, 0.24859331713001928, 0.24039556770542192, 0.24328840015335387, 0.24999021734633503], [-0.04226101813275727, 0.04274789309989058, 0.009490517261714827, 0.011269530514800695, -0.0034061250831855516, -0.029007677827829093, 0.04665914067479125, 0.022673213016811397, 0.0006165

In [42]:
# we can calculate the SNRs but we need to pay attention to the number and order of parameters
# the order is mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal, detector, Λ_1, Λ_2

snrs = SNR(PhenomD_NRTidal(), mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, network, Λ_1, Λ_2, auto_save=true, name_folder = "BNS");

Computing SNRs... 100%|██████████████████████████████████| Time: 0:00:02


SNRs computed!
The evaluation took: 2.234671601 seconds.


In [43]:
# same as before we can calculate the Fisher matrices
# we also introduce another detector in the network
# the network is now [CE_1, CE_2, ET, my_detector]
my_network = [CE_1, CE_2, ET, my_detector]
Fisher_BNS, SNRs_BNS = FisherMatrix(PhenomD_NRTidal(), mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal, my_network, Λ_1, Λ_2, return_SNR = true,
                                     auto_save=true, name_folder = "BNS/with_my_detector");
# note that I added to the name_folder "with_my_detector" to create a new folder

Computing Fishers and SNRs... 100%|██████████████████████| Time: 0:01:06


Fisher matrices and SNRs computed!
The evaluation took: 1.1014072317 minutes.


([0.0 0.0 … 0.0 0.0; 7.484486337768091e10 -4.296962409091215e9 … -3397.77885218351 -1059.938511145977; … ; 0.0 0.0 … 0.0 0.0; 4.540874143499499e10 2.490115047580765e9 … -1282.9781627931686 -1394.617006082412;;; 0.0 0.0 … 0.0 0.0; -4.296962409091215e9 2.537540903031452e8 … 267.10451412594324 78.17875586354606; … ; 0.0 0.0 … 0.0 0.0; 2.490115047580765e9 3.4251217840330505e8 … 183.7260025778242 168.815359503488;;; 0.0 0.0 … 0.0 0.0; -1.6433421822507887e9 9.588977412310112e7 … 88.88201851143288 26.715905067384092; … ; 0.0 0.0 … 0.0 0.0; -7.608460646226815e8 -3.926721943492921e7 … 24.91660018479325 26.70987546972814;;; … ;;; 0.0 0.0 … 0.0 0.0; 5.056620706675217e6 -289153.4907907401 … -0.22323206671674745 -0.07005710192256612; … ; 0.0 0.0 … 0.0 0.0; 3.1918524734773976e6 176641.3423884597 … -0.08759006827893429 -0.09546851745188578;;; 0.0 0.0 … 0.0 0.0; -3397.77885218351 267.10451412594324 … 0.0029115012228518936 0.0006988657778287607; … ; 0.0 0.0 … 0.0 0.0; -1282.9781627931686 183.7260025778

In [44]:
# To read the Fisher matrices and the SNRs you can use the following function
Fishers, SNRs = _read_Fishers_SNRs("output/BNS/with_my_detector/Fishers_SNRs.h5")

Attributes: ["Detectors", "What_this_file_contains", "number_events"]
Detectors: ["CE1Id", "CE2NM", "ETS", "mydetector"]
What_this_file_contains: This file contains the Fishers and the SNRs for 1000 events obtained with the PhenomD_NRTidal waveform model. The calculations are performed with the CE1Id,CE2NM,ETS,mydetector detectors and the correction due to Earth Motion was false.
number_events: 1000
Keys: ["Fishers", "SNRs"]


([0.0 0.0 … 0.0 0.0; 7.484486337768091e10 -4.296962409091215e9 … -3397.77885218351 -1059.938511145977; … ; 0.0 0.0 … 0.0 0.0; 4.540874143499499e10 2.490115047580765e9 … -1282.9781627931686 -1394.617006082412;;; 0.0 0.0 … 0.0 0.0; -4.296962409091215e9 2.537540903031452e8 … 267.10451412594324 78.17875586354606; … ; 0.0 0.0 … 0.0 0.0; 2.490115047580765e9 3.4251217840330505e8 … 183.7260025778242 168.815359503488;;; 0.0 0.0 … 0.0 0.0; -1.6433421822507887e9 9.588977412310112e7 … 88.88201851143288 26.715905067384092; … ; 0.0 0.0 … 0.0 0.0; -7.608460646226815e8 -3.926721943492921e7 … 24.91660018479325 26.70987546972814;;; … ;;; 0.0 0.0 … 0.0 0.0; 5.056620706675217e6 -289153.4907907401 … -0.22323206671674745 -0.07005710192256612; … ; 0.0 0.0 … 0.0 0.0; 3.1918524734773976e6 176641.3423884597 … -0.08759006827893429 -0.09546851745188578;;; 0.0 0.0 … 0.0 0.0; -3397.77885218351 267.10451412594324 … 0.0029115012228518936 0.0006988657778287607; … ; 0.0 0.0 … 0.0 0.0; -1282.9781627931686 183.7260025778

## BONUS: Waveforms

In [48]:
# If you are interested in just the waveforms of the events, you can use the functions
# Ampl (for the amplitude) and Phi (for the phase)

# the functions return the amplitude and the phase of the waveform, the order of the parameters is the same as always
# you also need to provide a frequency (array or float), the function will return the amplitude and the phase at the frequencies of the array (or float)
f = [10.0, 20.0, 30.0, 40.0, 50.0] # frequencies in Hz
amplitude = Ampl(PhenomD(), f, mc[1], η[1], χ_1[1], χ_2[1], dL[1])
phase = Phi(PhenomD(), f, mc[1], η[1], χ_1[1], χ_2[1])

5-element Vector{Float64}:
      0.0
 -10600.802858084211
 -13000.648224718958
 -13946.878140461731
 -14423.598699830402

In [52]:
# For PhenomHM() the functions Ampl() and Phase() return a structure with the amplitude for the different higher modes and the phase for the different higher modes

amplitude_HM = Ampl(PhenomHM(), f, mc[1], η[1], χ_1[1], χ_2[1], dL[1])
phase_HM = Phi(PhenomHM(), f, mc[1], η[1], χ_1[1], χ_2[1])

println(fieldnames(typeof(amplitude_HM))) # the structure has the fields :two_one, :two_two, :three_two, :three_three, :four_three, :four_four, representing L"\ell,m"

(:two_one, :two_two, :three_two, :three_three, :four_three, :four_four)


In [53]:
# To obtain the full waveform you can use the function hphc() which returns the plus and cross polarizations

h_plus, h_cross = hphc(PhenomHM(), f, mc[1], η[1], χ_1[1], χ_2[1], dL[1], ι[1])
# pay attention that you ned to provide the inclination angle (in radians)

(ComplexF64[5.99036707119903e-25 + 2.0576725683238865e-27im, 1.2676422785396177e-25 + 2.3216737655306055e-25im, 1.220790962866864e-25 + 1.1110763524369196e-25im, -2.5342586753912452e-26 - 1.155184120352223e-25im, -7.66362793296494e-26 - 4.7757892367418854e-26im], ComplexF64[-1.1661923531246423e-27 + 3.8300544439046686e-25im, -1.4852935690107672e-25 + 8.114658588313569e-26im, -7.109359313919804e-26 + 7.816716017808888e-26im, 7.391977089422936e-26 - 1.6278296680085194e-26im, 3.0517130859135547e-26 - 4.9044901052474e-26im])